***Import necessary libraries***

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

***Data Pre-Processing***

In [2]:
df = pd.read_csv('/content/spam.csv', delimiter=',', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
X = df.v2
Y = df.v1
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [6]:
tokenizer = Tokenizer(num_words=2000, lower=True)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen=200)

***Create Model***

In [7]:
model  = Sequential()

***Add layers***

In [8]:
model.add(Embedding(2000, 50, input_length=200))
model.add(LSTM(64))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
_________________________________________________________________


***Compile the Model***

In [10]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])


***Fit the Model***

In [11]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)


Epoch 1/10
28/28 [==============================] - 14s 367ms/step - loss: 0.3216 - accuracy: 0.8755 - val_loss: 0.1560 - val_accuracy: 0.9350
Epoch 2/10
28/28 [==============================] - 10s 344ms/step - loss: 0.0953 - accuracy: 0.9787 - val_loss: 0.0429 - val_accuracy: 0.9865
Epoch 3/10
28/28 [==============================] - 10s 345ms/step - loss: 0.0400 - accuracy: 0.9885 - val_loss: 0.0312 - val_accuracy: 0.9944
Epoch 4/10
28/28 [==============================] - 10s 344ms/step - loss: 0.0277 - accuracy: 0.9910 - val_loss: 0.0297 - val_accuracy: 0.9933
Epoch 5/10
28/28 [==============================] - 10s 364ms/step - loss: 0.0180 - accuracy: 0.9947 - val_loss: 0.0318 - val_accuracy: 0.9922
Epoch 6/10
28/28 [==============================] - 10s 345ms/step - loss: 0.0153 - accuracy: 0.9955 - val_loss: 0.0287 - val_accuracy: 0.9944
Epoch 7/10
28/28 [==============================] - 10s 345ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.0304 - val_accuracy: 0.9933

***Save the Model***

In [12]:
model.save("model.h5")

***Test the Model***

In [13]:
test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences, maxlen=200)
acc = model.evaluate(X_test, y_test)

35/35 [==============================] - 1s 28ms/step - loss: 0.0836 - accuracy: 0.9857


In [14]:
def predict(message):
    txt = tokenizer.texts_to_sequences(message)
    txt = sequence.pad_sequences(txt, maxlen=200)
    preds = model.predict(txt)
    if preds > 0.5:
        print("Spam")
    else:
        print("Not Spam")

In [15]:
predict(["Sorry, I'll call after the meeting."])

1/1 [==============================] - 1s 555ms/step
Not Spam


In [16]:
predict(["Congratulations!!! You won $50,000. Send message LUCKY100 to XXXXXXXXXX to recieve your prize."])


1/1 [==============================] - 0s 28ms/step
Spam


In [17]:
predict(["you won rupess 10,0000"])

1/1 [==============================] - 0s 24ms/step
Spam


In [18]:
predict(["This is the very important problem"])

1/1 [==============================] - 0s 24ms/step
Not Spam
